# Introduction to Graph Datbases

The first part of this assignment is designed to give you hands-on experience with graph databases. You will start by setting up an in-memory graph database, for which the support code is already written. Once the database is running, you will execute queries of increasing complexity, exploring how relationships between nodes and edges are stored and retrieved. Through this process, you will gain practical insights into graph database concepts such as connectivity, traversal, and querying using graph-specific languages.

In [19]:
import os
import sys
import pandas as pd
pd.set_option('display.max_colwidth', 200)

sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))
from utils import setup_database, download_sample_data

In [20]:
# Download sample data for the Kuzudb example
data_dir = '../data'
download_sample_data(data_dir, urls=[
    "https://kuzudb.com/data/movie-lens/movies.csv",
    "https://kuzudb.com/data/movie-lens/users.csv",
    "https://kuzudb.com/data/movie-lens/ratings.csv",
    "https://kuzudb.com/data/movie-lens/tags.csv"
])

# Set up the Kuzudb database connection
connection = setup_database('../tmp', delete_existing=True)

# Create schema
connection.execute('CREATE NODE TABLE Movie (movieId INT64, year INT64, title STRING, genres STRING, PRIMARY KEY (movieId))')
connection.execute('CREATE NODE TABLE User (userId INT64, PRIMARY KEY (userId))')
connection.execute('CREATE REL TABLE Rating (FROM User TO Movie, rating DOUBLE, timestamp INT64)')
connection.execute('CREATE REL TABLE Tags (FROM User TO Movie, tag STRING, timestamp INT64)')

# Insert data
connection.execute(f'COPY Movie FROM "{data_dir}/movies.csv" (HEADER=TRUE)')
connection.execute(f'COPY User FROM "{data_dir}/users.csv" (HEADER=TRUE)')
connection.execute(f'COPY Rating FROM "{data_dir}/ratings.csv" (HEADER=TRUE)')
connection.execute(f'COPY Tags FROM "{data_dir}/tags.csv" (HEADER=TRUE)')


Saved https://kuzudb.com/data/movie-lens/movies.csv to ../data/movies.csv
Saved https://kuzudb.com/data/movie-lens/users.csv to ../data/users.csv
Saved https://kuzudb.com/data/movie-lens/ratings.csv to ../data/ratings.csv
Saved https://kuzudb.com/data/movie-lens/tags.csv to ../data/tags.csv
Sample data downloaded successfully
Loading graph database
Removing existing database at ../tmp


## Running Queries

Now that your graph database is set up, you can begin querying it. This section includes seven queries, each increasing in complexity.

In [21]:
# Query 1: Query all nodes with the label 'Movie'. Return those movie nodes. Limit your results to 25
result = connection.execute("MATCH (m:Movie) RETURN m LIMIT 25")

df = result.get_as_df()
df.head()

,m
0,"{'_id': {'offset': 0, 'table': 0}, '_label': 'Movie', 'movieId': 833, 'year': 1996, 'title': 'High School High (1996)', 'genres': 'Comedy'}"
1,"{'_id': {'offset': 1, 'table': 0}, '_label': 'Movie', 'movieId': 835, 'year': 1996, 'title': 'Foxfire (1996)', 'genres': 'Drama'}"
2,"{'_id': {'offset': 2, 'table': 0}, '_label': 'Movie', 'movieId': 836, 'year': 1996, 'title': 'Chain Reaction (1996)', 'genres': 'Action|Adventure|Thriller'}"
3,"{'_id': {'offset': 3, 'table': 0}, '_label': 'Movie', 'movieId': 837, 'year': 1996, 'title': 'Matilda (1996)', 'genres': 'Children|Comedy|Fantasy'}"
4,"{'_id': {'offset': 4, 'table': 0}, '_label': 'Movie', 'movieId': 838, 'year': 1996, 'title': 'Emma (1996)', 'genres': 'Comedy|Drama|Romance'}"


In [22]:
# Query 2: Query all nodes with the label 'Movie'. Get all connected nodes to the movie nodes. Limit your results to 50
result = connection.execute(" MATCH (:Movie)--(p) RETURN p LIMIT 50")

df = result.get_as_df()
df.head()

,p
0,"{'_id': {'offset': 0, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 1}"
1,"{'_id': {'offset': 0, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 1}"
2,"{'_id': {'offset': 0, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 1}"
3,"{'_id': {'offset': 0, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 1}"
4,"{'_id': {'offset': 0, 'table': 1}, '_label': 'User', 'movieId': None, 'year': None, 'title': None, 'genres': None, 'userId': 1}"


In [23]:
# Query 3: Count the total number of nodes in the database
# Hint: Use the `COUNT` function to count the number of nodes
result = connection.execute(" MATCH (m) RETURN COUNT(m) AS total_nodes")

df = result.get_as_df()
df.head()

,total_nodes
0,10352


In [24]:
# Query 4: Query all nodes with the label 'User'. Count the degree for these nodes. Filter the nodes where the user rated more than 3 movies. Return the users and the degree
# Hint: First find all users and their ratings, then count the degree, and finally filter the results to only include users with more than 3 ratings
result = connection.execute(" MATCH (u:User)-[r:Rating]-(m:Movie) WITH u, COUNT(u) AS degree WHERE degree > 3 RETURN u, degree")

df = result.get_as_df()
df.head()

,u,degree
0,"{'_id': {'offset': 598, 'table': 1}, '_label': 'User', 'userId': 599}",2478
1,"{'_id': {'offset': 608, 'table': 1}, '_label': 'User', 'userId': 609}",37
2,"{'_id': {'offset': 607, 'table': 1}, '_label': 'User', 'userId': 608}",831
3,"{'_id': {'offset': 110, 'table': 1}, '_label': 'User', 'userId': 111}",646
4,"{'_id': {'offset': 520, 'table': 1}, '_label': 'User', 'userId': 521}",40


In [25]:
# Query 5: Query all nodes with the label 'Movie'. Each node has a 'genre' attribute. Count the number of nodes per genre
# Hint: Use the `WITH` clause to group by genres and count the number of movies
result = connection.execute("MATCH (m:Movie) WITH m.genres AS genre, COUNT(m) AS count RETURN genre, count")

df = result.get_as_df()
df.head()

,genre,count
0,Comedy,950
1,Action|Adventure|Thriller,40
2,Action|Comedy|Crime,42
3,Action|Adventure|Mystery|Romance|Thriller,1
4,Adventure|Children|Fantasy|Musical,2


In [26]:
# Query 6: Query all nodes with the label 'Movie' and 'User', and the edge 'Rating' between movie and user. Each edge 'Rating' has a rating. Find the top 10 rated movies by average rating score
# Hint: Use the AVG clause to calculate an average. Use the `ORDER BY` clause to sort the movies by rating in descending order
result = connection.execute("MATCH (u:User)-[r:Rating]-(m:Movie) WITH m, AVG(r.rating) AS AvgRating RETURN m.title, AvgRating ORDER BY AvgRating DESC LIMIT 10")

df = result.get_as_df()
df.head()

,m.title,AvgRating
0,Faster (2010),5.0
1,Tom and Jerry: A Nutcracker Tale (2007),5.0
2,Buzzard (2015),5.0
3,What Love Is (2007),5.0
4,Sorority House Massacre II (1990),5.0


In [27]:
# Query 7: Query all nodes with the label 'Movie' and 'User', and the edge 'Rating' between movie and user. Find pairs of movies often rated by the same users

result = connection.execute("""
MATCH (m1:Movie)-[r1:Rating]-(u1:User), (m2:Movie)-[r2:Rating]-(u1:User) 
WHERE m1 < m2
WITH m1, m2, COUNT(u1) AS commonUsers
ORDER BY commonUsers DESC
LIMIT 10
RETURN m1.title, m2.title, commonUsers
""")

df = result.get_as_df()
df.head()

,m1.title,m2.title,commonUsers
0,"Shawshank Redemption, The (1994)",Forrest Gump (1994),231
1,Pulp Fiction (1994),Forrest Gump (1994),230
2,Pulp Fiction (1994),"Shawshank Redemption, The (1994)",222
3,Pulp Fiction (1994),"Silence of the Lambs, The (1991)",207
4,Forrest Gump (1994),"Silence of the Lambs, The (1991)",199
